In [1]:

import functools
import math

import jax
import jax.numpy as jnp
from jax.experimental import pallas as pl
from jax.experimental.pallas import tpu as pltpu

def unrolled_fori_loop(length, body_fn, init, unroll):
 def unrolled_body(i, c):
   i *= unroll
   for j in range(unroll):
     c = body_fn(i + j, c)
   return c
 carry = jax.lax.fori_loop(0, length // unroll, unrolled_body, init)
 for j in range(length % unroll):
   carry = body_fn((length // unroll) * unroll + j, carry)
 return carry

def blockwise_topk(
  logits,
  k,
  block_topk_val=None,
  block_topk_index=None,
  start_k=0,
  num_blocks=128,
  mode='jax',
):
  '''Compute blockwise top-k.'''
  ntokens = logits.shape[0]

  if start_k != 0 and (block_topk_val is None or block_topk_index is None):
    raise ValueError(
      'start_k must be 0, unless precomputed top-(start_k) in a buffer is provided in block_topk_val and block_topk_index'
    )
  if mode == 'jax':
    block_topk_val = [
      jnp.full((ntokens, num_blocks), float('-inf'), dtype=logits.dtype)
      for i in range(k)
    ]
    # TODO?: Could use uint16 when vocab size < 4M if hardware supports
    block_topk_index = [
      jnp.full((ntokens, num_blocks), -1, dtype=jnp.int32) for i in range(k)
    ]
  elif mode == 'pallas':
    if block_topk_val is None or block_topk_index is None:
      raise ValueError(
        'Pass through of block_topk_val and tok_index expected for pallas topk.'
      )

  def while_body(i, while_carry):
    block_topk_val, block_topk_index = while_carry

    if mode == 'pallas':
      vals_carry = logits[..., pl.dslice(num_blocks * i, num_blocks)]
    elif mode == 'jax':
      vals_carry = jax.lax.dynamic_slice_in_dim(
        logits, num_blocks * i, num_blocks, axis=1
      )
    else:
      raise ValueError(
        'mode must be either `pallas` and a memory ref or `jax` and an array'
      )

    index_carry = jnp.full((ntokens, num_blocks), i, jnp.int32)

    for i in range(k):
      if i < start_k:
        # Nothing will be exchanged into the completed block topk, we just need
        # to invalidate it from flowing downward. So we check if it's already
        # found and invalidate if so.
        vals_carry = jnp.where(
          index_carry == block_topk_index[i], float('-inf'), vals_carry
        )
      else:
        # Sinking sort
        mask = vals_carry > block_topk_val[i]
        # TODO: Consider packing bfloat16 val and uint16 index into single uint32
        # and packed sort as in
        # https://github.com/triton-lang/triton/blob/main/python/triton_kernels/triton_kernels/topk_details/_topk_forward.py
        block_topk_val[i], vals_carry = (
          jnp.where(v, vals_carry, block_topk_val[i]) for v in (mask, ~mask)
        )
        block_topk_index[i], index_carry = (
          jnp.where(v, index_carry, block_topk_index[i]) for v in (mask, ~mask)
        )
    return (block_topk_val, block_topk_index)
  return unrolled_fori_loop(logits.shape[-1] // num_blocks, while_body, (block_topk_val, block_topk_index), unroll=16
  )

# Pallas kernel scaffolding
def topk_blockwise_superset_kernel(
  logits_ref,
  block_topm_val_ref,
  block_topm_index_ref,
  max_m_ref,
  flag_ref,
  num_blocks: int = 128,
  k: int = 64,
  m_schedule: tuple[int] | None = None,
):
  '''Compute blockwise top-m's until they contain global top-k.'''
  ### Initialize refs
  shape = block_topm_val_ref.shape
  block_topm_val_ref[...] = jnp.full(shape, float('-inf'), dtype=logits_ref.dtype)
  block_topm_index_ref[...] = jnp.full(shape, -1, dtype=jnp.int32)

  block_token = logits_ref.shape[0]
  for i in range(block_token):
    # Worst case m = k
    max_m_ref[pl.program_id(0) * block_token + i] = k

  # flag for termination of while loop
  flag_ref[0] = 0

  ### Run increasing block topk, until sure overall topk present
  if m_schedule is None:
    m_schedule = (5, 8, 12)
  # Ensure worst case of all k in one block is covered
  m_schedule = (0,) + m_schedule + (k,)

  for completed_m, m in zip(m_schedule, m_schedule[1:]):

    @pl.when(flag_ref[0] == 0)
    def _():
      topk_vals, topk_indexs = blockwise_topk(
        logits_ref,
        # bf16, bf16 -> i1 mask not supported on v5e so we cast to f32
        # TODO: check v6e, bf16 comparitor and make model specific
        block_topk_val=[block_topm_val_ref[:,pl.dslice(i*num_blocks, num_blocks)].astype(jnp.float32) for i in range(m)],
        block_topk_index=[block_topm_index_ref[:,pl.dslice(i*num_blocks, num_blocks)] for i in range(m)],
        k=m,
        start_k=completed_m,
        mode='pallas',
      )

      for i in range(completed_m, m):
        block_topm_val_ref[:,pl.dslice(i*num_blocks, num_blocks)] = topk_vals[i].astype(block_topm_val_ref.dtype)
        block_topm_index_ref[:,pl.dslice(i*num_blocks,num_blocks)] = topk_indexs[i].astype(
          block_topm_index_ref.dtype
        )

      # Stopping criterion check
      # To find top-k values of a set, we can split into N subsets,
      # and sort the largest, 2nd-largest, 3-rd largest, ..., m-th largest values for each subset
      # When in the superset of top-(m-1) subsets there are more than k values
      # larger (or equal than) the largest m'th largest value from the subsets
      # then the top-(m-1) subsets must contain the top-k of the set.
      # We run a schedule of m's until we have that full top-k found.
      pivot_point = topk_vals[m - 1].max(-1, keepdims=True)
      n_larger = (
        sum([(v >= pivot_point) for v in topk_vals[: m - 1]])
        .astype(jnp.float32)
        .sum(-1)
      )
      # flag SMEM used to check if all searches terminated
      flag_ref[0] = 0
      for i in range(block_token):
        blockwise_topm_contains_topk = n_larger[i] >= k
        flag_ref[0] += blockwise_topm_contains_topk
        # Store when the criteria was hit for each query
        token_index = pl.program_id(0) * block_token + i
        max_m = max_m_ref[token_index]
        max_m_ref[token_index] = jnp.where(
          blockwise_topm_contains_topk & (max_m == k), m - 1, max_m
        )

      # If not all terminated, reset the flag say we need to search deeper
      @pl.when(flag_ref[0] != block_token)
      def _():
        flag_ref[0] = 0


# Pallas function
def topk_blockwise_superset_pallas(
  logits, k, num_blocks=128, block_token=None, m_schedule=None
):
  num_tokens, vocab_size = logits.shape
  if block_token is None:
    block_token = min(32, num_tokens)
  if num_tokens % block_token != 0:
    raise ValueError('token block size must be a multiple of num tokens')

  out_shape = (
    jax.ShapeDtypeStruct((num_tokens, k*num_blocks), logits.dtype),
    # uint16 fits vocab size of up to 2**16 * 128 = 8.4M. But not used to avoid unforseen issues.
    jax.ShapeDtypeStruct((num_tokens, k*num_blocks), jnp.int32),
    jax.ShapeDtypeStruct((num_tokens,), jnp.int32),
    jax.ShapeDtypeStruct((1,), jnp.int32),  # scratch for termination flag
  )
  out_specs = jax.tree.map(
    lambda _: pl.BlockSpec((block_token, k*num_blocks), lambda i: (i, 0)), out_shape[:2]
  )
  out_specs += (
    pl.BlockSpec(memory_space=pltpu.SMEM),
    pl.BlockSpec(memory_space=pltpu.SMEM),
  )
  return pl.pallas_call(
    functools.partial(
      topk_blockwise_superset_kernel,
      k=k,
      num_blocks=num_blocks,
      m_schedule=m_schedule,
    ),
    in_specs=(pl.BlockSpec((block_token, vocab_size), lambda i: (i, 0)),),
    out_shape=out_shape,
    grid=(num_tokens // block_token),
    out_specs=out_specs,
    compiler_params=pltpu.TPUCompilerParams(vmem_limit_bytes=2**26),
  )(logits)

gather2d = jax.vmap(lambda x, y: x[y])
def dense_gather_kernel(val_ref, index_ref, out_ref, num_lanes=128):
  index = index_ref[...]
  out = jnp.zeros(index_ref.shape, val_ref.dtype)
  for i in range(0, val_ref.shape[1], num_lanes):
    mask = (index >= i) & (index < i + num_lanes)
    out = jnp.where(
      mask,
      gather2d(
          val_ref[:, pl.dslice(i, num_lanes)],
          index % num_lanes),
      out
    )
  out_ref[...] = out.astype(out_ref.dtype)

def dense_gather_pallas(val, index, out_dtype=None, num_lanes=128):
 if out_dtype is None:
   out_dtype = val.dtype
 return pl.pallas_call(
   dense_gather_kernel,
   out_shape=jax.ShapeDtypeStruct(index.shape, out_dtype),
   in_specs=(
     pl.BlockSpec((8,val.shape[1]), lambda i: (i,0)),
     pl.BlockSpec((8, num_lanes), lambda i:(i,0)),
   ),
   out_specs = pl.BlockSpec((8, num_lanes), lambda i: (i,0)),
   grid = (val.shape[0] // 8,),
 )(val, index)


def topk_on_filtered_subset(block_topm_val, block_topm_index, k):
  num_blocks = 128
  topk_logits, local_indices = jax.lax.top_k(
    block_topm_val, k=k
  )

  block_topm_flat_index = (block_topm_index * num_blocks) + (jax.lax.broadcasted_iota(block_topm_index.dtype, block_topm_index.shape, 1) % num_blocks)
  topk_flat_indices = dense_gather_pallas(
    block_topm_flat_index,
    jnp.concatenate([
      local_indices,
      jnp.zeros((local_indices.shape[0], num_blocks - k), local_indices.dtype)
    ], axis=-1),
  )[:,:k]
  return topk_logits, topk_flat_indices


@functools.partial(
  jax.jit,
  static_argnames=(
    'k',
    'num_blocks',
    'm_stage1_schedule',
    'm_stage2_schedule',
    'block_token',
  ),
)
def topk_optimized(
  logits,
  k: int = 64,
  num_blocks: int = 128,
  m_stage1_schedule: tuple[int] | None = None,
  m_stage2_schedule: tuple[int] | None = None,
  block_token: int | None = None,
):
  '''Fast implementation of jax.lax.top_k on TPUs.'''
  if logits.ndim != 2:
    raise ValueError('Expected 2D input')
  block_topm_val, block_topm_index, termination_m, _ = topk_blockwise_superset_pallas(
    logits,
    k=k,
    block_token=block_token,
    m_schedule=m_stage1_schedule,
    num_blocks=num_blocks,
  )

  # top-k the smallest number of values we can, by taking max m required
  # such that all queries to have full top-k
  # We compile for a range of shapes, then use jax.lax.cond to run just one.
  # in practice 8 nearly always sufficient
  if m_stage2_schedule is None:
    m_init = 8
    m_stage2_schedule = [
      m_init * (2**i) for i in range(int(math.log2(k // m_init)) + 1)
    ]
  # Guarantee all cases covered
  m_stage2_schedule = (-1,) + tuple(m_stage2_schedule) + (k,)

  # Buffer for output to be written in to
  topk_logits, topk_flat_indices = jax.tree.map(
    jnp.zeros_like,
    topk_on_filtered_subset(block_topm_val, block_topm_index, k=k),
  )
  max_m = termination_m.max()
  for lower_m, upper_m in zip(m_stage2_schedule, m_stage2_schedule[1:]):
    topk_logits, topk_flat_indices = jax.lax.cond(
      (max_m > lower_m) & (max_m <= upper_m),
      lambda *args: topk_on_filtered_subset(
        block_topm_val=block_topm_val[:,:num_blocks*upper_m],
        block_topm_index=block_topm_index[:num_blocks*upper_m],
        k=k,
      ),
      lambda *args: args,
      topk_logits,
      topk_flat_indices,
    )
  return topk_logits, topk_flat_indices

import functools
import jax
import jax.numpy as jnp

k = 64
num_queries = 32
vocab_size = 201088
hidden_dim = 2880

logit_key, key_act, key_weight = jax.random.split(jax.random.key(0), 3)
x = jax.random.normal(key_act, (num_queries, hidden_dim), dtype=jnp.bfloat16)
w = jax.random.normal(key_weight, (hidden_dim, vocab_size), dtype=jnp.bfloat16)
logits = jax.random.normal(key_weight, (num_queries, vocab_size), dtype=jnp.float32).astype(jnp.bfloat16)

topk_xla = jax.jit(jax.lax.top_k, static_argnames=('k',))
approx_topk_xla = jax.jit(jax.lax.approx_max_k, static_argnames=('k',))

@jax.jit
@functools.partial(jax.vmap, in_axes=(0, None))
def matmul_and_topk_xla(x, w, k=k):
  logits = (x @ w)
  return jax.lax.top_k(logits, k)

def run():
  # reference runtimes
  #o = jax.block_until_ready(x @ w)
  #jax.block_until_ready(matmul_and_topk_xla(x, w))
  jax.block_until_ready(topk_xla(logits, k=k))

  # Optimized tpu run on random logits
  jax.block_until_ready(topk_optimized(logits, k=k))

  # Not exact. Runtime varies with recall, here run with default 0.95
  jax.block_until_ready(approx_topk_xla(logits, k=k))



run()
with jax.profiler.trace('/content/'):
  run()


import gzip
from glob import glob
import json
import pandas as pd
import os
path = sorted(glob('/content/plugins/profile/*/**.json.gz'), key=os.path.getmtime)[-1]
print(path)
trace = json.load(gzip.open(path))
df = pd.DataFrame(trace['traceEvents'])
df = df[~df.name.isna()]
df[df.name.str.contains('jit_topk_optimized')].iloc[0].dur



/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


/content/plugins/profile/2025_10_02_00_36_03/c718fa664a9c.trace.json.gz


np.float64(76.451172)

In [ ]:
jax.lax.top_k(logits, k), topk_optimized(logits, k)

([Array([[4.4375, 4.28125, 4.1875, ..., 3.42188, 3.40625, 3.40625],
         [4.59375, 4.46875, 4.25, ..., 3.45312, 3.4375, 3.4375],
         [4.65625, 4.53125, 4.15625, ..., 3.45312, 3.45312, 3.4375],
         ...,
         [4.5625, 4.5, 4.34375, ..., 3.39062, 3.39062, 3.39062],
         [4.34375, 4.3125, 4.09375, ..., 3.4375, 3.42188, 3.42188],
         [4.53125, 4.40625, 4.0625, ..., 3.40625, 3.40625, 3.40625]],      dtype=bfloat16),
  Array([[101171,  58512,  64088, ..., 141062,  15207, 116690],
         [ 39735,  63692, 112073, ..., 186850,  22206,  70731],
         [192142,  55046, 153857, ..., 137158, 142069,   7672],
         ...,
         [125137,  79273,  25911, ...,  20426,  66520,  99947],
         [ 56456, 126343,  96079, ..., 100138,  28483, 162640],
         [ 22603, 114276,  15680, ...,  89922,  91875,  92696]],      dtype=int32)],
 (Array([[4.4375, 4.28125, 4.1875, ..., 3.42188, 3.40625, 3.40625],
         [4.59375, 4.46875, 4.25, ..., 3.45312, 3.4375, 3.4375],
       